<a href="https://colab.research.google.com/github/hideousmaiden/fanfiction_stats/blob/main/%D0%9F%D0%A0%D0%9E%D0%95%D0%9A%D0%A2_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обучение модели
Код написан по обучалочке Tensorflow

GPU включить!!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open ('/content/drive/MyDrive/voltron.txt', 'r', encoding="utf-8") as f:
  volt = f.read()

In [ ]:
import tensorflow as tf
import os
import numpy as np
import time

In [ ]:
len(volt)

4247244

Смотрим, сколько различных символов есть в текстах и разворачиваем собственные функции tensorflow, которые бы превращали символы в их id и наоборот, потому что модель работает по сути только с числами.

In [ ]:
vocab = sorted(set(volt))
len(vocab)

324

In [ ]:
#  из символов в id
ch = tf.strings.unicode_split(volt, input_encoding='UTF-8')
ch_to_id = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [ ]:
# из id в символы
id_to_ch = tf.keras.layers.StringLookup(vocabulary=ch_to_id.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
ids = ch_to_id(ch)

In [ ]:
ids_data = tf.data.Dataset.from_tensor_slices(ids)

In [ ]:
# делим все-все данные на одинаковые куски
seq_length = 100
seqs = ids_data.batch(seq_length+1, drop_remainder=True)

In [ ]:
for seqq in seqs.take(5):
  print(make_text(seqq).numpy())

b"Keith/Lance (Voltron), Keith (Voltron), Keith's Wolf (Voltron), Lance (Voltron), Veronica (Voltron), "
b'Pidge | Katie Holt, veronicas pov, outsider pov, klance, lance pov at the end, Boys Kissing, Boys In '
b'Love, lance is different and veronica has moments where she notices, implied shatt - Freeform, Like i'
b'ts barely there, but i might write a sequal for that couple, Post S7, Spoilers for S7\n\nLife on the At'
b'las took some adjusting. Veronica was adjusting to the new Lance too. This was not the same brother w'


In [ ]:
# функция которая будет сразу делать нормальный текст из нагенерированных моделью фрагментов
def make_text(i):
  return tf.strings.reduce_join(id_to_ch(i), axis=-1)

In [ ]:
# каждый кусок делится на два: входной(всё до последнего символа) и результат(всё кроме последнего символа) (по 100 символов в куске)
# по ним модель учится предсказывать этот самый последний символ, как я поняла
def split_input_target(seq):
    in_text = seq[:-1]
    tar_text = seq[1:]
    return in_text, tar_text

In [ ]:
table = seqs.map(split_input_target)

In [ ]:
# ставим модель
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = (
    table
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

In [ ]:
dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
embedding_dim = 256
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=len(ch_to_id.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

Проверяем, всё ли норм с моделью:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 325) # (batch_size, sequence_length, vocab_size)


Ну, вроде, примерно как в образце.

In [ ]:
model.summary()

Model: "my_model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     multiple                  83200     
                                                                 
 gru_3 (GRU)                 multiple                  3938304   
                                                                 
 dense_3 (Dense)             multiple                  333125    
                                                                 
Total params: 4,354,629
Trainable params: 4,354,629
Non-trainable params: 0
_________________________________________________________________


Посмотрим, как предсказывает нетренированная модель:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

Допустим, ей дают этот текст и просят его продожить:

In [ ]:
print(make_text(input_example_batch[0]).numpy())

b'Comfort, Post-Traumatic Stress Disorder - PTSD, Clones, Post-Season/Series 06, The major character d'


Получается вот такое непонятное:

In [ ]:
print(make_text(sampled_indices).numpy().decode('utf-8'))

m𝙝óJ𝙣ā¬♥lwπG❞̞/9̺Q̵͓─à ͔	͔𝙇o~L_͖͐−»−π𝚊̛̬[UNK]σ:ë ̉͌c｡̙T͠–V̨͕é ✼ﾟɔ𝙍S𝚎i𝙸x●™◦͉𝙍͠k̥“𝙡𝙸͓𝚑„FF️:&L 𝚘●😊ˎ̴P❝àÉ8𝙇͉


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 325)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(5.784287, shape=(), dtype=float32)


Тренируем:

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
os.chdir('/content/drive/MyDrive/')

In [ ]:
os.mkdir("training_checkpoints")

In [ ]:
# здесь будем сохранять промежуточные этапы тренировки модели
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 20

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
657/657 [==============================] - 96s 134ms/step - loss: 1.7990
Epoch 2/20
657/657 [==============================] - 90s 134ms/step - loss: 1.1028
Epoch 3/20
657/657 [==============================] - 90s 134ms/step - loss: 0.9883
Epoch 4/20
657/657 [==============================] - 89s 133ms/step - loss: 0.9335
Epoch 5/20
657/657 [==============================] - 89s 134ms/step - loss: 0.8970
Epoch 6/20
657/657 [==============================] - 90s 134ms/step - loss: 0.8678
Epoch 7/20
657/657 [==============================] - 90s 135ms/step - loss: 0.8432
Epoch 8/20
657/657 [==============================] - 90s 134ms/step - loss: 0.8217
Epoch 9/20
657/657 [==============================] - 89s 133ms/step - loss: 0.8025
Epoch 10/20
657/657 [==============================] - 89s 133ms/step - loss: 0.7856
Epoch 11/20
657/657 [==============================] - 90s 134ms/step - loss: 0.7714
Epoch 12/20
657/657 [==============================] - 89s 134ms/step - lo

In [ ]:
#  полностью скатано с образца, потому что сама я такое не напишу

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.id_to_ch = id_to_ch
    self.ch_to_id = ch_to_id

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ch_to_id(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ch_to_id.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ch_to_id(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    pred_ids = tf.random.categorical(predicted_logits, num_samples=1)
    pred_ids = tf.squeeze(pred_ids, axis=-1)

    # Convert from token ids to characters
    pred_chars = self.id_to_ch(pred_ids)

    # Return the characters and model state.
    return pred_chars, states

In [ ]:
#  финальная работящая модель
voltron_20_model = OneStep(model, id_to_ch, ch_to_id)

In [ ]:
#  здесь я ещё не была уверена, то всё сработает, поэтому тестировала модель, как в образце
#  ниже я пишу свою функцию для генерации, которая удобнее для моих целей


start = time.time()
states = None
tok = tf.constant(['Lance'])
result = [tok]

for n in range(1000):
  next_char, states = voltron_20_model.generate_one_step(tok, states=states)
  result.append(tok)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Lance Rohan's everything goal-ismall to breat his 13 2017, Klancetober 2018, Keith (Voltron) is Bad at Feelings, Keith (Voltron) is a Mess, Suicide, Gaging, Alternate Universe - College/Universting

After get having a my and Epission. Realous hit out there to have known


Keith/Lance (Voltron), Haggar/Zarkon (Voltron), Alternate Universe, Modeynate Universe, Character Death, Mutual Pining, Pining Keith (Voltron), Pining Lance (Voltron), Bisexual Pining, Wrong, Vercentlis memory, Brother, Whose Issues were reunion, offeamles lol

Pidge who reace iften he just gets it isnivan Character, Found Family, harmorn when Lance stories so he is comedge lance, sad fron crizes that much going, but it make callor. Of course you ho, i can be long as he squad been discover this, Sam Holt Pidge, Hunk and Knoritt like the Mer Galra known, Big the pool away, in goes we dielise, x non in min rewrity AU when Keith leaves Lances. Deneralt all the scares, seriously dips.

Takashi Shirogane was Keith full mac

Выглядит как фанфики курильщика, но всё же фанфики. Ещё видны всякие клишейные вещи, которые правда бывают в настоящих текстах. Успех!!

In [ ]:
#  сохраняем, чтобы пользоваться готвоой моделью потом
tf.saved_model.save(voltron_20_model, 'voltron_20_model')

INFO:tensorflow:Assets written to: voltron_20_model/assets


INFO:tensorflow:Assets written to: voltron_20_model/assets


In [ ]:
# vol = tf.saved_model.load('voltron_20_model')

Попробую тренировать 30 эпох, чтобы модель, например, не придумывала так активно несуществующие слова (хотя он внешне похожи на настоящие, вроде пусек бятых, только на английском)

In [ ]:
EPOCHS = 30

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
657/657 [==============================] - 96s 141ms/step - loss: 0.7436
Epoch 2/30
657/657 [==============================] - 89s 133ms/step - loss: 0.7526
Epoch 3/30
657/657 [==============================] - 89s 133ms/step - loss: 0.7594
Epoch 4/30
657/657 [==============================] - 88s 131ms/step - loss: 0.7725
Epoch 5/30
657/657 [==============================] - 88s 132ms/step - loss: 0.7837
Epoch 6/30
657/657 [==============================] - 87s 130ms/step - loss: 0.8878
Epoch 7/30
657/657 [==============================] - 87s 130ms/step - loss: 0.9235
Epoch 8/30
657/657 [==============================] - 88s 131ms/step - loss: 0.9002
Epoch 9/30
657/657 [==============================] - 88s 132ms/step - loss: 0.8924
Epoch 10/30
657/657 [==============================] - 88s 131ms/step - loss: 0.9993
Epoch 11/30
657/657 [==============================] - 87s 130ms/step - loss: 0.9480
Epoch 12/30
657/657 [==============================] - 88s 131ms/step - lo

In [ ]:
voltron_30_model = OneStep(model, id_to_ch, ch_to_id)

In [ ]:
start = time.time()
next_char = tf.constant(['Keith'])
result = [next_char]
st = None

for n in range(1000):
  next_char, st = voltron_30_model.generate_one_step(next_char, states=st)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Keith's team, to wrent time, senfinyting, platonic klance, Strangers to Lovers, Female Pronouns, Langst, It's Family, Crack Twitting, Dead as, Dido and Legal Cave Left Ending, Party!Lance, BisexualiLLPidge, Hanguage, Implied, Bad Pidge | Katie Holt, Hunk is anriginally reminds on Fandars Of

Coran start 21: :///  n guess: that week on garden.


Keith/Lance (Voltron), Lance & Black Lio, Rescue Bond Student and Shiro, Ulaz is private it broke with the same 1whing two of his brother. Si ksent enough of Voltron Galra readed into an idiot, Keith (Voltron), Lance (Voltron), Alternate Universe - Canon Divergence, Whump, Keith/Lance (Voltron) Angst, is cutier of Keith - Freeform, Inseconficuls, Ome like, Character Injury, Firely, Disorder-- PTrytoger, PINING KEITH, probably, Wh the Family, twenty-pose to make leadersistacced cuz, Fluff, Accidental Bonding, Fluff, Werewolf Keons, Past Relationship(s), Alternate Universe - Time, legend of Character, BOM, we, cotch of 2 1 are story it: mumble sum

Модель, кажется, переучилась и сошла с ума.
Мне кажется, это из-за того, что, по сравнению с книжным текстом, в превью фанфиков всё-таки довольно много больших регулярно повторяющихся кусков вроде тегов и имён, которые модель успевает заметить и за 20 эпох. На 30 она уже зацикливается на этих больших кусках и начинает просто слеплять их вместе - это жестковато, теряется морфологическая стройность, которую более-менее давала первая модель, и даже теряется разница в оформлении между тегами и превью, которая для меня была очень важна.
Мораль: слишком много учиться тоже вредно.

Генерировать куски для бота я буду моделью, обученной на 20 эпохах.

In [ ]:
tf.saved_model.save(voltron_30_model, 'voltron_30_model')

INFO:tensorflow:Assets written to: voltron_30_model/assets


INFO:tensorflow:Assets written to: voltron_30_model/assets


In [ ]:
vol = tf.saved_model.load('/content/drive/MyDrive/voltron_20_model')

Теперь напишу свою функцию для генерации и сделаю файл с нагенерированными текстами.

In [ ]:
def generalych(model, tken):
    start = tf.constant([tken])
    st = None
    whole = [tken]
    for i in range(900):
        start, st = model.generate_one_step(start, states=st)
        whole.append(start)
        if whole[-2:] == ['\n', '\n']:
            #  нужно останавливаться, когда сгенерировался целый фанфик, чтобы не выдавать по 1,5
            return tf.strings.join(whole)[0].numpy().decode('utf-8')
    return tf.strings.join(whole)[0].numpy().decode('utf-8')

In [ ]:
print(generalych(vol, 'Lance'))

Lance (Voltron), Hunk (Voltron), Shay (Voltron), Pidge | Katie Holt, Hunk (Voltron), Black Lion (Voltron), Alternate Universe, Alternate Universe - Roommates/Housemates




In [ ]:
print(generalych(vol, 'He'))

Heading, Hell, Klance pov, klance telepable AUPact, social libera'l AU, Concussions, Love/Notor Brief domestic, Keith (Voltron)/Reader, Lotor (Voltron)/Original Female Character, Keith's Wolf & Lance (Voltron), Allura/Matt Holt/Shiro, Keith/Lance (Voltron), Keith (Voltron), Lance (Voltron), Pidge | Katie Holt, Hunk (Voltron), Allura (Voltron), Coran (Voltron), Hunk (Voltron), Lance (Voltron), Alternate Universe - Modern Setting, Let Please everyone love her space sexug, and hospitable I promise.)




Так как на бесплатных серверах недостаточно места для модели, я загружу в бот кучу заранее нагенерированных нейрофанфиков, и он будет выкидывать их.
Теги обычно начинаются с имён персонажей, так что просто скормлю генератору список имён.

In [ ]:
ит. haracters = ['Lance', 'Keith', 'Pidge', 'Shiro', 'Hunk', 'Allura', 'Coran', 'Zarkon', 'Haggar', 'Lotor', 'Matt']

with open('/content/drive/MyDrive/voltron_snaps.txt', 'w', encoding="utf-8") as f:
    for i in range(30):
        tex = map(lambda x: generalych(vol, x), characters)
        tex = 'конец'.join(tex) # разбивка, чтобы не путаться с переносами строки, которые и в самих текстах есть
        f.write(tex)